In [1]:
import pandas as pd
import requests
import json
import math
from time import sleep

token = ''
url = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'

df_geom = pd.DataFrame()

COLUNAS = [
    'SOLICITACAO',
	'TIPO',
	'ORGAO',
	'DATA_ASS',
	'HORARIO',
	'ASSUNTO',
	'SUBDIVISAO',
	'DESCRICAO',
	'LOGRADOURO_ASS',
	'CEP_ASS',
	'NUMERO_ASS',
	'BAIRRO_ASS',
	'REGIONAL_ASS',
	'MEIO_RESPOSTA',
	'OBSERVACAO',
	'SEXO',
	'BAIRRO_CIDADAO',
	'REGIONAL_CIDADAO',
	'DATA_NASC',
	'TIPO_CIDADAO',
	'ORGAO_RESP',
	'RESPOSTA_FINAL',
    'GEOM'
]

# Gera um geom vazio para evitar 
df_geom = pd.DataFrame(columns=COLUNAS)

df_resultado = pd.read_csv('Resultado/inundacao156.csv', sep=';')
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27843 entries, 0 to 27842
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        27843 non-null  int64  
 1   SOLICITACAO       27843 non-null  float64
 2   TIPO              27843 non-null  object 
 3   ORGAO             22142 non-null  object 
 4   DATA_ASS          27843 non-null  object 
 5   HORARIO           27843 non-null  object 
 6   ASSUNTO           27843 non-null  object 
 7   SUBDIVISAO        27843 non-null  object 
 8   DESCRICAO         27843 non-null  object 
 9   LOGRADOURO_ASS    27843 non-null  object 
 10  CEP_ASS           27843 non-null  int64  
 11  NUMERO_ASS        27843 non-null  float64
 12  BAIRRO_ASS        27843 non-null  object 
 13  REGIONAL_ASS      27843 non-null  object 
 14  MEIO_RESPOSTA     27843 non-null  object 
 15  OBSERVACAO        13270 non-null  object 
 16  SEXO              27843 non-null  object

In [ ]:
# Retirei as linhas onde não haviam reginal nem o numero do endereço da solicitacao
df_resultado = df_resultado.dropna(subset=['REGIONAL_ASS'])
df_resultado = df_resultado.dropna(subset=['NUMERO_ASS'])

In [3]:
for index, row in df_resultado.iterrows():
    rua = row['LOGRADOURO_ASS']
    numero = row['NUMERO_ASS']
    uri = f'{rua},{numero},curitiba.json?access_token={token}'

    req = requests.get(f'{url}{uri}')
    if req.status_code != 200:
        print(f"Requisição mal sucedida! {row['SOLICITACAO']}")
        
    content = req.content    
    my_new_string_value = content.decode("utf-8")
    my_json = json.loads(my_new_string_value)
    
    new_register = {
        'SOLICITACAO':math.floor(row['SOLICITACAO']),
        'TIPO':row['TIPO'],
        'ORGAO':row['ORGAO'],
        'DATA_ASS':row['DATA_ASS'],
        'HORARIO':row['HORARIO'],
        'ASSUNTO':row['ASSUNTO'],
        'SUBDIVISAO':row['SUBDIVISAO'],
        'DESCRICAO':row['DESCRICAO'],
        'LOGRADOURO_ASS':row['LOGRADOURO_ASS'],
        'CEP_ASS':row['CEP_ASS']*10,
        'NUMERO_ASS':math.floor(row['NUMERO_ASS']),
        'BAIRRO_ASS':row['BAIRRO_ASS'].upper(),
        'REGIONAL_ASS':row['REGIONAL_ASS'].upper(),
        'MEIO_RESPOSTA':row['MEIO_RESPOSTA'],
        'OBSERVACAO':row['OBSERVACAO'],
        'SEXO':row['SEXO'],
        'BAIRRO_CIDADAO':row['BAIRRO_CIDADAO'],
        'REGIONAL_CIDADAO':row['REGIONAL_CIDADAO'],
        'DATA_NASC':row['DATA_NASC'],
        'TIPO_CIDADAO':row['TIPO_CIDADAO'],
        'ORGAO_RESP':row['ORGAO_RESP'],
        'RESPOSTA_FINAL':row['RESPOSTA_FINAL'],
        'GEOM':my_json['features'][0]["geometry"]["coordinates"]
    }
    
    df_geom = df_geom.append(new_register, ignore_index=True)
    sleep(0.01)

In [5]:
df_geom.to_csv('Resultado/inundacao156latlng.csv', sep=';', mode = 'a', header = False, index=False)